In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
im = pd.read_csv("Datasets/immigrants.csv", skiprows=14, nrows=215)
im.columns=['Birthplace', '1980-1990', '1991-2000', '2001-2010', '2011-2021']

In [3]:
for i in range(len(im)):
    if im['Birthplace'][i][-2:].isdigit():
        im['Birthplace'][i] = im['Birthplace'][i][0:-3]
    if im['Birthplace'][i][-1].isdigit():
        im['Birthplace'][i] = im['Birthplace'][i][0:-2]

    im['1980-1990'][i] = int(im['1980-1990'][i].replace(',', ''))
    im['1991-2000'][i] = int(im['1991-2000'][i].replace(',', ''))
    im['2001-2010'][i] = int(im['2001-2010'][i].replace(',', ''))
    im['2011-2021'][i] = int(im['2011-2021'][i].replace(',', ''))
    if im['1980-1990'][i] == 0 and im['1991-2000'][i] == 0 and im['2001-2010'][i] == 0 and im['2011-2021'][i] == 0:
        im=im.drop(labels=i)
im = im.sort_values(by=['Birthplace'])
im.reset_index(drop=True, inplace=True)
im

,Birthplace,1980-1990,1991-2000,2001-2010,2011-2021
0,Afghanistan,3920,14745,22280,21340
1,Albania,15,4130,8000,5500
2,Algeria,1345,11745,32430,32580
3,Andorra,0,0,0,30
4,Angola,415,390,1315,580
...,...,...,...,...,...
195,"Virgin Islands, United States",10,0,20,15
196,West Bank and Gaza Strip,1065,1930,2375,2620
197,Yemen,120,785,1120,2570
198,Zambia,480,685,780,845


In [28]:
gdp = pd.read_csv("Datasets/gdp.csv", skiprows=4, nrows=266, usecols=[0, 55, 65])
gdp.columns = ['Birthplace', 'GDP2011', 'GDP2021']
life = pd.read_csv("Datasets/life.csv", skiprows=4, usecols=[55, 64])
life.columns = ['LE2011', 'LE2020']
inflation = pd.read_csv("Datasets/inflation.csv", skiprows=4, usecols=[55, 63])
inflation.columns = ['IF2011', 'IF2019']
unemployment = pd.read_csv("Datasets/unemployment.csv", skiprows=4, usecols=[56, 64])
unemployment.columns = ['UE2012', 'UE2020']
health = pd.read_csv("Datasets/health.csv", skiprows=4, usecols=[55, 63])
health.columns = ['Health2011', 'Health2019']
education = pd.read_csv("Datasets/education.csv", skiprows=4, usecols=[55, 64])
education.columns = ['EDU2011', 'EDU2020']

In [29]:
comb = pd.concat([gdp, life, inflation, unemployment, health, education], axis=1)
comb.dropna(axis=0,subset = ['GDP2011', 'GDP2021', 'LE2011', 'LE2020', 'IF2011', 'IF2019', 'UE2012', 'UE2020', 'Health2011', 'Health2019', 'EDU2011', 'EDU2020'], inplace=True)
comb = comb.sort_values(by=['Birthplace'])
comb.reset_index(drop=True, inplace=True)
comb

,Birthplace,GDP2011,GDP2021,LE2011,LE2020,IF2011,IF2019,UE2012,UE2020,Health2011,Health2019,EDU2011,EDU2020
0,Afghanistan,621.912311,368.754614,61.419000,62.575000,11.804186,2.302373,1.690000,11.710000,8.561907,13.242202,3.46201,2.860859
1,Armenia,3462.681611,4966.513471,73.305000,72.173000,7.650008,1.443447,17.299999,12.180000,9.375878,11.335264,3.14385,2.705670
2,Australia,62596.432175,60443.109165,81.895122,83.200000,3.303850,1.610768,5.220000,6.460000,8.541538,9.910786,5.06995,6.100000
3,Austria,51442.276246,53637.705711,80.982927,81.192683,3.286579,1.530896,4.910000,5.200000,10.026133,10.434031,5.59227,5.070000
4,Azerbaijan,7189.691229,5387.997975,69.984000,66.868000,7.858333,2.610572,5.190000,7.160000,2.446418,4.038290,2.43564,4.332050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Upper middle income,7546.030310,10828.050694,74.458433,75.845915,5.005595,1.968230,5.092209,6.962978,5.349713,5.844136,4.22994,4.134130
93,Uruguay,14267.584306,17313.188348,76.983000,78.430000,8.092832,7.881989,6.450000,10.330000,7.948956,9.347832,4.35527,4.552070
94,Uzbekistan,2051.129515,1983.064723,69.649000,70.331000,13.779745,14.525736,4.870000,5.290000,5.143769,5.617604,6.19586,4.922790
95,Vietnam,1953.556979,3756.489121,73.692000,75.378000,18.677732,2.795824,1.030000,2.100000,4.612852,5.249656,4.81372,4.111617


In [34]:
def increase_rate(num1, num2):
    return (num2 - num1) / num1

a, b, c, d, e, f, g, h = [], [], [], [], [], [], [], []
for m in range(len(comb)):
    for n in range(len(im)):
        if comb['Birthplace'][m] == im['Birthplace'][n]:
            a.append(im['Birthplace'][n])
            b.append(im['2011-2021'][n])
            c.append(increase_rate(comb['GDP2011'][m], comb['GDP2021'][m]))
            d.append(comb['LE2020'][m] - comb['LE2011'][m])
            e.append(increase_rate(comb['IF2011'][m], comb['IF2019'][m]))
            f.append(increase_rate(comb['UE2012'][m], comb['UE2020'][m]))
            g.append(increase_rate(comb['Health2011'][m], comb['Health2019'][m]))
            h.append(increase_rate(comb['EDU2011'][m], comb['EDU2020'][m]))

data_1121 = pd.DataFrame({'Birthplace':a, 'Immigration':b, 'GDP_Change':c, 'LE_Change':d, 'IF_Change': e, 'UE_Change': f, 'HE_Change': g, 'EE_Change': h})
data_1121

,Birthplace,Immigration,GDP_Change,LE_Change,IF_Change,UE_Change,HE_Change,EE_Change
0,Afghanistan,21340,-0.407063,1.156000,-0.804953,5.928994,0.546642,-0.173642
1,Armenia,2120,0.434297,-1.132000,-0.811314,-0.295954,0.208982,-0.139377
2,Austria,785,0.042678,0.209756,-0.534198,0.059063,0.040684,-0.093391
3,Azerbaijan,1905,-0.250594,-3.116000,-0.667796,0.379576,0.650695,0.778608
4,Belarus,2850,0.118747,3.673171,-0.894828,7.100000,0.202478,0.057134
...,...,...,...,...,...,...,...,...
61,Trinidad and Tobago,4095,-0.104846,1.128000,-0.804089,0.131720,0.486680,-0.044758
62,Ukraine,26470,0.305209,0.375854,-0.008674,0.258964,0.041435,-0.093536
63,Uruguay,580,0.213463,1.447000,-0.026053,0.601550,0.175982,0.045187
64,Uzbekistan,2250,-0.033184,0.682000,0.054137,0.086242,0.092118,-0.205471


In [36]:
X = data_1121[['Immigration', 'GDP_Change', 'LE_Change', 'IF_Change', 'UE_Change', 'HE_Change', 'EE_Change']]
km = KMeans(n_clusters=5, n_init=10).fit(X)
pd.DataFrame(a, km.labels_)


,0
4,Afghanistan
0,Armenia
0,Austria
0,Azerbaijan
0,Belarus
...,...
0,Trinidad and Tobago
4,Ukraine
0,Uruguay
0,Uzbekistan
